In [1]:
import numpy as np
import cv2
import tensorflow as tf
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from numpy.random import randn
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Attention
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.layers import Add
import tensorflow.keras.backend as K
from matplotlib import pyplot
from tensorflow.keras.utils import plot_model

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 11GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=11264)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# define the discriminator model
def define_discriminator(image_shape, vector_shape, n_classes=7):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # source image input
    in_src_image = Input(shape=image_shape)
    # vector input
    in_target = Input(shape=(vector_shape[0], vector_shape[1]))
    # label input
    in_label = Input(shape=(1,))
    # Vector layer
    n_nodes = (image_shape[0] * image_shape[1] * image_shape[2])
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # scale up to image dimensions with linear
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((image_shape[0], image_shape[1], image_shape[2]))(li)
    # LSTM Model
    p = LSTM(200, activation='relu')(in_target)
    p = Dense(n_nodes)(p)
    # Reshape and mergo to image dimension
    p = Reshape((image_shape[0], image_shape[1], image_shape[2]))(p)
    merged = Concatenate()([p, in_src_image, li])
    # concatenate images channel-wise
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Flatten()(d)
    d = Dropout(0.4)(d)
    out_layer = Dense(1, activation='sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target, in_label], out_layer)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [4]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add downsampling layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same',
    kernel_initializer=init)(layer_in)
    # conditionally add batch normalization
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g

In [5]:
# define the standalone generator model
def define_generator(in_shape, vector_shape, latent_dim, n_classes=7):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=in_shape)
    in_lat = Input(shape=latent_dim)
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    li = Dense(latent_dim)(li)
    li = Reshape([latent_dim])(li)
    gen = LeakyReLU(alpha=0.2)(in_lat)
    # merge image gen and label input
    # encoder model
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    b = Flatten()(b)
    # Merge latent space variable input with network
    b = Concatenate()([b, gen, li])
    # Reshape to vector size
    n_nodes = (vector_shape[0] * vector_shape[1])
    b = Dense(n_nodes)(b)
    b = Reshape([vector_shape[0], vector_shape[1]])(b)
    # encoder-decoder LSTM model
    d = LSTM(200, activation='relu', return_sequences=True)(b)
    d = LSTM(200, activation='relu')(d)
    d = Dense(n_nodes)(d)
    out_layer = Reshape([vector_shape[0], vector_shape[1]])(d)
    # output


    # define model input & output
    model = Model([in_image, in_lat, in_label], out_layer)
    return model

In [6]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape, vector_shape, latent_dim):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    in_lat = Input(shape=latent_dim)
    in_label = Input(shape=(1,))
    # connect the source image to the generator input
    gen_out = g_model([in_src, in_lat, in_label])
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out, in_label])
    # src image as input, generated image and classification output
    model = Model([in_src, in_lat, in_label], [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [7]:
# load and prepare training images
def load_real_samples(filename):
	# load compressed arrays
	data = load(filename)
	# unpack arrays
	X1, X2, X3 = data['arr_0'], data['arr_1'], data['arr_2']
	return [X1, X2, X3-1]

In [8]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples):
	# unpack dataset
	img, vec, label = dataset
	# choose random instances
	ix = randint(0, img.shape[0], n_samples)
	# retrieve selected images
	X1, X2, X3 = img[ix], vec[ix], label[ix]
	# generate 'real' class labels 
	y = ones((n_samples, 1))
	return [X1, X2, X3], y

In [9]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [10]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, label, n_samples):
    # Generate latent space points
    z_input = generate_latent_points(latent_dim, n_samples)
    # generate fake instance
    X = g_model.predict([samples, z_input, label])
    # create 'fake' class labels 
    y = zeros((n_samples, 1))
    return X, y

In [11]:
# Overlay trajectories (data_y) to the image (data_x)
def create_trajectory(data_x, data_y, obs_len=10):
    # Calibration parameter to overlay for a 1280x360 resolution image
    K = np.array([[537.023764, 0, 640 , 0], 
                    [0 , 537.023764, 180, 0], 
                    [0, 0, 1, 0]])
    # Rotation matrix to obtain egocentric trajectory
    Rt = np.array([[0.028841, 0.007189, 0.999558, 1.481009],
                    [-0.999575,  0.004514,  0.028809,  0.296583],
                    [ 0.004305,  0.999964, -0.007316, -1.544537],
                    [ 0.      ,  0.      ,  0.      ,  1.      ]])

    # Resize data back to 1280x360
    data_x = cv2.resize(data_x, (1280,360))
    # Add column of ones for rotation matrix multiplication
    data_y = np.hstack((data_y, np.ones((len(data_y),1))))
    # Draw points
    for m in range(obs_len, data_y.shape[0]):
        # Rotation matrix multiplication of trajectory 
        A = np.matmul(np.linalg.inv(Rt), data_y[m, :].reshape(4, 1))
        # Egocentric view of trajectory
        B = np.matmul(K, A)
        # Circle location of trajectories 
        x = int(B[0, 0] * 1.0 / B[2, 0])
        y = int(B[1, 0] * 1.0 / B[2, 0])
        if (x < 0 or x > 1280 - 1 or y > 360 - 1):
            continue
        # Use opencv to overlay trajectories
        data_x = cv2.circle(data_x, (x, y), 3, (0, 0, 255), -1)
    return data_x

In [12]:
def generate_drvact_text(drvact):
    text = '[warnings] drvact label is not defined ...'
    if (drvact == 1):
        text = 'Go'
    elif (drvact == 2):
        text = 'Turn Left'
    elif (drvact == 3):
        text = 'Turn Right'
    elif (drvact == 4):
        text = 'U-turn'
    elif (drvact == 5):
        text = 'Left LC'
    elif (drvact == 6):
        text = 'Right LC'
    elif (drvact == 7):
        text = 'Avoidance'
    return text

In [13]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, dataset, n_samples=1):
    # select a sample of input images
    [X_real_img, X_real_vec, X_label], _ = generate_real_samples(dataset, n_samples)
    # generate a batch of fake samples
    X_fake_vec, _ = generate_fake_samples(g_model, X_real_img, X_label, n_samples)
    # scale all pixels from [-1,1] to [0,1]
    X_real_img = (X_real_img + 1) / 2.0
    pyplot.figure(figsize=(32.0, 20.0))
    # plot real source images
    for i in range(n_samples):
        orig_image = (X_real_img[i]* 255).astype(np.uint8)
        orig_image = cv2.resize(orig_image, (1280,360))
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(orig_image)
    # plot generated target image
    for i in range(n_samples):
        fake_sample = create_trajectory((X_real_img[i]* 255).astype(np.uint8), X_fake_vec[i])
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(fake_sample)
    # plot real target image
    for i in range(n_samples):
        true_sample = create_trajectory((X_real_img[i]* 255).astype(np.uint8), X_real_vec[i])
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(true_sample)
    # save plot to file
    filename1 = 'plot_%06d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = 'model_%06d.h5' % (step+1)
    g_model.save(filename2)
    filename3 = 'd_model_%06d.h5' % (step+1)
    d_model.save(filename3)
    print('>Saved: %s, %s and %s' % (filename1, filename2, filename3))

In [14]:
# train models
def train(d_model, g_model, gan_model, dataset, latent_dim, n_epochs=100, batch_size=30):
    # iteration number to output checkpoint
    display_iter = 1000
    # calculate the number of batches
    n_batch = int(len(dataset[0]) / batch_size)
    # calculate the number of training iterations
    n_steps = n_batch * n_epochs
    # manually enumerate epochs
    for i in range(n_steps):
        # select a batch of real samples
        [X_real_img, X_real_vec, X_label], y_real = generate_real_samples(dataset, batch_size)
        # generate a batch of fake samples
        X_fake_vec, y_fake = generate_fake_samples(g_model, X_real_img, X_label, batch_size)
        # update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_real_img, X_real_vec, X_label], y_real)
        # update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_real_img, X_fake_vec, X_label], y_fake)
        X_lat = generate_latent_points(latent_dim, batch_size)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch([X_real_img, X_lat, X_label], [y_real, X_real_vec])
        # summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # summarize model performance
        #if (i+1) % int(bat_per_epo) == 0:
        if (i+1) % int(display_iter) == 0:
            summarize_performance(i, g_model, dataset)
            print('saved')

In [15]:
%%time
# load image data
dataset = load_real_samples('./dual_condition_dataset_train.npz')
print('Loaded', dataset[0].shape, dataset[1].shape, dataset[2].shape)

Loaded (16805, 256, 256, 3) (16805, 40, 3) (16805,)
CPU times: user 23.7 s, sys: 7.04 s, total: 30.7 s
Wall time: 31 s


In [16]:
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
vector_shape = dataset[1].shape[1:]
latent_dim = 512
# %%
# define the models
d_model = define_discriminator(image_shape, vector_shape)
g_model = define_generator(image_shape, vector_shape, latent_dim)
# %%
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape, vector_shape, latent_dim)

In [17]:
#see model summary
print(g_model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 128, 128, 64) 3136        input_4[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_7 (LeakyReLU)       (None, 128, 128, 64) 0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 64, 64, 128)  131200      leaky_re_lu_7[0][0]              
____________________________________________________________________________________________

In [ ]:
# train model
train(d_model, g_model, gan_model, dataset, latent_dim)

>1, d1[0.387] d2[1.610] g[386.886]
>2, d1[0.060] d2[0.535] g[335.865]
>3, d1[0.057] d2[0.435] g[416.619]
>4, d1[0.112] d2[0.327] g[370.783]
>5, d1[0.071] d2[0.342] g[372.602]
>6, d1[0.102] d2[0.273] g[387.435]
>7, d1[0.100] d2[0.198] g[333.936]
>8, d1[0.081] d2[0.150] g[350.727]
>9, d1[0.052] d2[0.116] g[368.737]
>10, d1[0.150] d2[0.116] g[257.483]
>11, d1[0.078] d2[0.068] g[341.842]
>12, d1[0.047] d2[0.057] g[353.030]
>13, d1[0.069] d2[0.107] g[327.447]
>14, d1[0.195] d2[0.192] g[285.190]
>15, d1[0.074] d2[0.043] g[331.264]
>16, d1[0.052] d2[0.020] g[362.561]
>17, d1[0.079] d2[0.015] g[320.245]
>18, d1[0.102] d2[0.023] g[274.505]
>19, d1[1.108] d2[0.144] g[317.370]
>20, d1[0.078] d2[0.164] g[275.581]
>21, d1[0.121] d2[0.064] g[280.394]
>22, d1[0.133] d2[0.085] g[268.016]
>23, d1[0.175] d2[0.013] g[283.859]
>24, d1[0.069] d2[0.020] g[354.795]
>25, d1[0.149] d2[0.025] g[261.318]
>26, d1[0.115] d2[0.020] g[244.353]
>27, d1[0.129] d2[0.029] g[249.984]
>28, d1[0.142] d2[0.014] g[236.646]
>

>227, d1[0.318] d2[0.417] g[86.611]
>228, d1[0.330] d2[0.386] g[83.597]
>229, d1[0.341] d2[0.359] g[94.650]
>230, d1[0.353] d2[0.353] g[97.601]
>231, d1[0.346] d2[0.398] g[98.662]
>232, d1[0.344] d2[0.396] g[89.687]
>233, d1[0.330] d2[0.367] g[90.152]
>234, d1[0.313] d2[0.375] g[101.111]
>235, d1[0.305] d2[0.398] g[88.445]
>236, d1[0.350] d2[0.392] g[100.784]
>237, d1[0.329] d2[0.381] g[113.473]
>238, d1[0.330] d2[0.358] g[99.918]
>239, d1[0.358] d2[0.356] g[91.580]
>240, d1[0.332] d2[0.356] g[95.162]
>241, d1[0.337] d2[0.373] g[95.205]
>242, d1[0.337] d2[0.356] g[93.146]
>243, d1[0.334] d2[0.255] g[149.487]
>244, d1[0.349] d2[0.180] g[105.537]
>245, d1[0.467] d2[0.354] g[79.507]
>246, d1[0.333] d2[0.388] g[116.621]
>247, d1[0.337] d2[0.356] g[84.194]
>248, d1[0.342] d2[0.327] g[81.683]
>249, d1[0.332] d2[0.347] g[85.141]
>250, d1[0.372] d2[0.361] g[97.902]
>251, d1[0.345] d2[0.382] g[88.522]
>252, d1[0.345] d2[0.340] g[115.000]
>253, d1[0.317] d2[0.358] g[88.349]
>254, d1[0.363] d2[0.

>454, d1[0.339] d2[0.350] g[90.625]
>455, d1[0.336] d2[0.358] g[77.578]
>456, d1[0.333] d2[0.353] g[83.423]
>457, d1[0.340] d2[0.372] g[70.262]
>458, d1[0.348] d2[0.367] g[76.031]
>459, d1[0.346] d2[0.364] g[76.101]
>460, d1[0.349] d2[0.346] g[87.149]
>461, d1[0.336] d2[0.347] g[100.744]
>462, d1[0.353] d2[0.340] g[108.808]
>463, d1[0.337] d2[0.351] g[65.718]
>464, d1[0.336] d2[0.349] g[85.105]
>465, d1[0.339] d2[0.365] g[91.282]
>466, d1[0.330] d2[0.351] g[67.730]
>467, d1[0.343] d2[0.353] g[62.270]
>468, d1[0.363] d2[0.354] g[71.141]
>469, d1[0.344] d2[0.340] g[74.139]
>470, d1[0.367] d2[0.347] g[70.184]
>471, d1[0.338] d2[0.348] g[80.962]
>472, d1[0.339] d2[0.358] g[78.470]
>473, d1[0.335] d2[0.359] g[65.203]
>474, d1[0.354] d2[0.339] g[89.793]
>475, d1[0.344] d2[0.365] g[75.026]
>476, d1[0.371] d2[0.366] g[99.159]
>477, d1[0.341] d2[0.355] g[101.554]
>478, d1[0.349] d2[0.349] g[75.036]
>479, d1[0.348] d2[0.331] g[87.962]
>480, d1[0.354] d2[0.358] g[63.966]
>481, d1[0.325] d2[0.352]

>682, d1[0.349] d2[0.328] g[77.420]
>683, d1[0.352] d2[0.340] g[82.616]
>684, d1[0.372] d2[0.338] g[62.929]
>685, d1[0.384] d2[0.333] g[76.194]
>686, d1[0.341] d2[0.350] g[66.094]
>687, d1[0.338] d2[0.333] g[65.728]
>688, d1[0.338] d2[0.315] g[94.909]
>689, d1[0.357] d2[0.328] g[76.281]
>690, d1[0.343] d2[0.342] g[75.270]
>691, d1[0.396] d2[0.334] g[71.423]
>692, d1[0.388] d2[0.333] g[73.667]
>693, d1[0.362] d2[0.352] g[65.606]
>694, d1[0.347] d2[0.322] g[92.069]
>695, d1[0.349] d2[0.341] g[71.683]
>696, d1[0.356] d2[0.348] g[59.244]
>697, d1[0.339] d2[0.357] g[63.267]
>698, d1[0.354] d2[0.349] g[58.139]
>699, d1[0.366] d2[0.342] g[61.553]
>700, d1[0.351] d2[0.342] g[51.625]
>701, d1[0.341] d2[0.385] g[69.933]
>702, d1[0.354] d2[0.346] g[67.982]
>703, d1[0.346] d2[0.359] g[69.190]
>704, d1[0.348] d2[0.353] g[58.479]
>705, d1[0.354] d2[0.350] g[75.809]
>706, d1[0.357] d2[0.364] g[70.140]
>707, d1[0.356] d2[0.355] g[77.299]
>708, d1[0.338] d2[0.340] g[63.866]
>709, d1[0.344] d2[0.341] g[

>910, d1[0.358] d2[0.331] g[56.474]
>911, d1[0.368] d2[0.314] g[58.310]
>912, d1[0.354] d2[0.358] g[57.612]
>913, d1[0.377] d2[0.327] g[47.263]
>914, d1[0.345] d2[0.330] g[64.318]
>915, d1[0.344] d2[0.328] g[63.177]
>916, d1[0.364] d2[0.326] g[60.229]
>917, d1[0.351] d2[0.326] g[64.359]
>918, d1[0.393] d2[0.318] g[69.205]
>919, d1[0.358] d2[0.270] g[79.473]
>920, d1[0.413] d2[0.306] g[62.466]
>921, d1[0.349] d2[0.335] g[58.895]
>922, d1[0.390] d2[0.319] g[62.910]
>923, d1[0.343] d2[0.317] g[61.409]
>924, d1[0.344] d2[0.321] g[55.625]
>925, d1[0.369] d2[0.331] g[53.672]
>926, d1[0.402] d2[0.323] g[66.118]
>927, d1[0.357] d2[0.297] g[61.192]
>928, d1[0.451] d2[0.299] g[56.178]
>929, d1[0.365] d2[0.284] g[68.068]
>930, d1[0.356] d2[0.258] g[65.534]
>931, d1[0.341] d2[0.221] g[70.718]
>932, d1[0.427] d2[0.208] g[88.729]
>933, d1[0.336] d2[0.158] g[84.093]
>934, d1[0.369] d2[0.123] g[89.993]
>935, d1[0.358] d2[0.048] g[69.109]
>936, d1[0.342] d2[0.129] g[63.884]
>937, d1[0.393] d2[0.089] g[

...

>3570, d1[0.333] d2[0.363] g[32.819]
>3571, d1[0.373] d2[0.334] g[33.766]
>3572, d1[0.362] d2[0.341] g[25.877]
>3573, d1[0.340] d2[0.374] g[27.326]
>3574, d1[0.322] d2[0.373] g[28.319]
>3575, d1[0.354] d2[0.343] g[35.501]
>3576, d1[0.349] d2[0.378] g[33.639]
>3577, d1[0.353] d2[0.386] g[29.343]
>3578, d1[0.338] d2[0.373] g[28.806]
>3579, d1[0.365] d2[0.372] g[35.052]
>3580, d1[0.334] d2[0.365] g[27.792]
>3581, d1[0.331] d2[0.378] g[32.836]
>3582, d1[0.323] d2[0.380] g[24.429]
>3583, d1[0.360] d2[0.354] g[42.529]
>3584, d1[0.336] d2[0.367] g[27.243]
>3585, d1[0.344] d2[0.372] g[31.513]
>3586, d1[0.332] d2[0.362] g[36.824]
>3587, d1[0.351] d2[0.358] g[26.158]
>3588, d1[0.361] d2[0.355] g[28.913]
>3589, d1[0.327] d2[0.332] g[33.186]
>3590, d1[0.325] d2[0.350] g[34.647]
>3591, d1[0.379] d2[0.330] g[28.705]
>3592, d1[0.373] d2[0.353] g[34.401]
>3593, d1[0.360] d2[0.354] g[31.916]
>3594, d1[0.355] d2[0.341] g[36.604]
>3595, d1[0.355] d2[0.344] g[33.505]
>3596, d1[0.323] d2[0.374] g[31.689]
>

>3792, d1[0.332] d2[0.378] g[26.929]
>3793, d1[0.307] d2[0.380] g[32.092]
>3794, d1[0.306] d2[0.380] g[30.643]
>3795, d1[0.400] d2[0.328] g[29.314]
>3796, d1[0.287] d2[0.347] g[32.849]
>3797, d1[0.350] d2[0.363] g[30.915]
>3798, d1[0.342] d2[0.342] g[34.659]
>3799, d1[0.300] d2[0.399] g[23.377]
>3800, d1[0.384] d2[0.326] g[30.945]
>3801, d1[0.360] d2[0.322] g[33.322]
>3802, d1[0.391] d2[0.349] g[28.532]
>3803, d1[0.368] d2[0.360] g[27.229]
>3804, d1[0.335] d2[0.357] g[29.820]
>3805, d1[0.337] d2[0.361] g[34.497]
>3806, d1[0.332] d2[0.374] g[30.356]
>3807, d1[0.387] d2[0.335] g[38.247]
>3808, d1[0.326] d2[0.360] g[40.891]
>3809, d1[0.355] d2[0.352] g[36.338]
>3810, d1[0.374] d2[0.337] g[37.461]
>3811, d1[0.383] d2[0.324] g[34.446]
>3812, d1[0.382] d2[0.342] g[37.062]
>3813, d1[0.333] d2[0.328] g[33.085]
>3814, d1[0.326] d2[0.347] g[37.959]
>3815, d1[0.400] d2[0.322] g[38.074]
>3816, d1[0.364] d2[0.338] g[21.574]
>3817, d1[0.339] d2[0.368] g[26.283]
>3818, d1[0.343] d2[0.367] g[28.021]
>

>4012, d1[0.359] d2[0.346] g[29.988]
>4013, d1[0.345] d2[0.332] g[26.146]
>4014, d1[0.339] d2[0.347] g[27.553]
>4015, d1[0.305] d2[0.326] g[33.834]
>4016, d1[0.304] d2[0.331] g[33.331]
>4017, d1[0.356] d2[0.329] g[23.679]
>4018, d1[0.319] d2[0.366] g[25.222]
>4019, d1[0.336] d2[0.357] g[22.938]
>4020, d1[0.309] d2[0.378] g[25.068]
>4021, d1[0.395] d2[0.319] g[24.075]
>4022, d1[0.306] d2[0.372] g[27.800]
>4023, d1[0.373] d2[0.321] g[25.955]
>4024, d1[0.335] d2[0.367] g[24.561]
>4025, d1[0.321] d2[0.352] g[27.203]
>4026, d1[0.305] d2[0.399] g[23.855]
>4027, d1[0.310] d2[0.357] g[38.307]
>4028, d1[0.337] d2[0.342] g[26.446]
>4029, d1[0.302] d2[0.359] g[39.081]
>4030, d1[0.338] d2[0.334] g[38.159]
>4031, d1[0.336] d2[0.356] g[29.486]
>4032, d1[0.404] d2[0.313] g[25.824]
>4033, d1[0.332] d2[0.366] g[29.171]
>4034, d1[0.302] d2[0.308] g[29.553]
>4035, d1[0.371] d2[0.343] g[23.520]
>4036, d1[0.344] d2[0.320] g[25.814]
>4037, d1[0.268] d2[0.440] g[23.551]
>4038, d1[0.335] d2[0.359] g[28.787]
>

>4234, d1[0.337] d2[0.330] g[39.651]
>4235, d1[0.328] d2[0.339] g[27.869]
>4236, d1[0.375] d2[0.314] g[33.082]
>4237, d1[0.358] d2[0.350] g[27.705]
>4238, d1[0.422] d2[0.302] g[20.083]
>4239, d1[0.304] d2[0.453] g[28.440]
>4240, d1[0.343] d2[0.328] g[26.384]
>4241, d1[0.337] d2[0.368] g[25.498]
>4242, d1[0.407] d2[0.326] g[27.541]
>4243, d1[0.346] d2[0.349] g[26.703]
>4244, d1[0.340] d2[0.351] g[29.056]
>4245, d1[0.341] d2[0.356] g[24.183]
>4246, d1[0.346] d2[0.394] g[29.801]
>4247, d1[0.353] d2[0.372] g[28.252]
>4248, d1[0.290] d2[0.387] g[19.369]
>4249, d1[0.344] d2[0.341] g[25.272]
>4250, d1[0.348] d2[0.332] g[27.802]
>4251, d1[0.268] d2[0.337] g[32.500]
>4252, d1[0.281] d2[0.412] g[26.556]
>4253, d1[0.359] d2[0.330] g[33.309]
>4254, d1[0.332] d2[0.391] g[27.912]
>4255, d1[0.349] d2[0.386] g[32.338]
>4256, d1[0.272] d2[0.461] g[25.706]
>4257, d1[0.348] d2[0.387] g[22.584]
>4258, d1[0.374] d2[0.315] g[22.268]
>4259, d1[0.391] d2[0.342] g[31.803]
>4260, d1[0.346] d2[0.341] g[33.303]
>

>4456, d1[0.364] d2[0.341] g[24.405]
>4457, d1[0.347] d2[0.346] g[28.353]
>4458, d1[0.311] d2[0.390] g[25.243]
>4459, d1[0.318] d2[0.389] g[22.730]
>4460, d1[0.347] d2[0.341] g[26.423]
>4461, d1[0.360] d2[0.340] g[22.537]
>4462, d1[0.351] d2[0.344] g[18.152]
>4463, d1[0.357] d2[0.346] g[30.669]
>4464, d1[0.333] d2[0.375] g[28.131]
>4465, d1[0.334] d2[0.335] g[27.141]
>4466, d1[0.365] d2[0.361] g[26.936]
>4467, d1[0.286] d2[0.360] g[35.412]
>4468, d1[0.307] d2[0.339] g[20.731]
>4469, d1[0.358] d2[0.337] g[29.335]
>4470, d1[0.353] d2[0.350] g[31.148]
>4471, d1[0.334] d2[0.366] g[29.893]
>4472, d1[0.339] d2[0.358] g[23.457]
>4473, d1[0.320] d2[0.365] g[31.220]
>4474, d1[0.373] d2[0.334] g[24.147]
>4475, d1[0.359] d2[0.351] g[23.974]
>4476, d1[0.345] d2[0.369] g[20.522]
>4477, d1[0.364] d2[0.345] g[20.557]
>4478, d1[0.323] d2[0.433] g[20.284]
>4479, d1[0.342] d2[0.377] g[26.686]
>4480, d1[0.316] d2[0.372] g[26.055]
>4481, d1[0.340] d2[0.356] g[26.917]
>4482, d1[0.356] d2[0.338] g[30.777]
>

In [ ]:
#end the tensorflow session
from tensorflow.keras import backend as K
K.clear_session()
#end the tensorflow session

In [ ]:
#for cuda to release GPU memory
from numba import cuda
cuda.select_device(0)
cuda.close()

# 